In [1]:
import time
import cv2
import numpy as np
import math

print("Load MobileNetSSD model")

prototxt_path = "MobileNetSSD_deploy.prototxt"
model_path = "MobileNetSSD_deploy.caffemodel"

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]

net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

pos_line=550
offset=50
car = 0


def center_object(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

def process_frame_MobileNetSSD(next_frame):
    global car
    
    rgb = cv2.cvtColor(next_frame, cv2.COLOR_BGR2RGB)
    (H, W) = next_frame.shape[:2]


    blob = cv2.dnn.blobFromImage(next_frame, size=(300, 300), ddepth=cv2.CV_8U)
    net.setInput(blob, scalefactor=1.0/127.5, mean=[127.5, 127.5, 127.5])
    detections = net.forward()

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.1:
            
            idx = int(detections[0, 0, i, 1])
            if CLASSES[idx] != "car":
                continue
                
            label = CLASSES[idx]

            box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
            (startX, startY, endX, endY) = box.astype("int")
            
            center_ob = center_object(startX, startY, endX-startX, endY-startY)
            cv2.circle(next_frame, center_ob, 4, (0, 0,255), -1)
            
            if center_ob[1]<(pos_line+offset) and center_ob[1]>(pos_line-offset):
                car+=1
            
            cv2.putText(next_frame, label+' '+str(round(confidence,2)), (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2)
            cv2.rectangle(next_frame, (startX, startY), (endX, endY), (0, 255, 0), 3)

    return next_frame

def VehicheDetection_UsingMobileNetSSD(filename):
    cap = cv2.VideoCapture(filename)
    global car

    frame_count = 0


    while True:
        ret, next_frame = cap.read() 
        
        if ret == False: break

        frame_count += 1
        cv2.line(next_frame, (25, pos_line), (1200, pos_line), (255,127,0), 3) 
        next_frame = process_frame_MobileNetSSD(next_frame)

        cv2.putText(next_frame, "Kendaraan Lewat : "+str(car), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
        cv2.imshow("Video Original" , next_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break


    print("/MobileNetSSD Car Detector")


    cap.release()
    cv2.destroyAllWindows()


filename = 'video.mp4'
VehicheDetection_UsingMobileNetSSD(filename)

Load MobileNetSSD model


KeyboardInterrupt: 